device-history-by-company-and-city-clickhouse

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()

In [ ]:
query_text = """--sql
-- Соединяем добавляем данные из справочников
SELECT 
    *
FROM db1.intercoms_st_asgard_ch AS intercoms_st_asgard_ch
						
limit 10
"""

ch.query_run(query_text)

In [47]:
query_text = """--sql
SELECT
    device_type,
    created_at,
    row_number() OVER (PARTITION BY device_uuid ORDER BY created_at) AS r_number
FROM db1.device_history_dir_partner
limit 2
"""
ch.query_run(query_text)

,device_type,created_at,r_number
0,intercom,2023-05-04 07:51:30,1
1,intercom,2024-10-29 09:06:03,1


In [ ]:
query_text = """--sql
SELECT
    device_uuid,
    device_type,
    created_at,
    if(company_uuid_from = 'b1782e4f-9198-49d1-b5aa-7bdba9c87d21' AND r_number = 1,1,0) AS first_appearance
FROM
    (SELECT
        device_type,
        device_uuid,
        company_uuid_from,
        company_uuid_to,
        created_at,
        row_number() OVER (PARTITION BY device_uuid ORDER BY created_at) AS r_number
FROM db1.device_history_dir_partner
) AS t1
limit 10
"""
ch.query_run(query_text)

,device_uuid,device_type,first_appearance
0,0001e60b-206f-4047-a5c0-79b6f06f22b2,intercom,1
1,00027edc-553c-44d0-82cb-a1f8b0a6b8f2,intercom,1
2,00027edc-553c-44d0-82cb-a1f8b0a6b8f2,intercom,0
3,0004ed9a-adc1-4d02-a4f5-9674baa53a40,intercom,1
4,000618b2-7dc8-4c45-a7ac-4ca3852cc14b,intercom,1
5,00090f57-e55d-4f99-be8e-91fda7ddbb0e,intercom,1
6,000adcd6-05cd-43a8-a3d0-4a4822434f22,intercom,1
7,000adcd6-05cd-43a8-a3d0-4a4822434f22,intercom,0
8,000adcd6-05cd-43a8-a3d0-4a4822434f22,intercom,0
9,000fe5f8-7d7b-4eba-abba-76e90d902573,intercom,1


In [2]:
query_text = """--sql
SELECT 
	intercoms_st_asgard_ch.report_date as report_date,
	intercoms_st_asgard_ch.intercom_uuid AS device_uuid,
	motherboard_id AS serial_number,
	'intercom' AS type,
	intercoms_st_asgard_ch.partner_uuid AS asgard_partner_uuid,
	intercoms_st_partner_ch.partner_uuid AS partner_uuid,
	intercoms_st_partner_ch.installation_point_id AS installation_point_id,
	intercoms_st_partner_ch.model_identifier as model_identifier
FROM db1.intercoms_st_asgard_ch AS intercoms_st_asgard_ch
LEFT JOIN db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard_ch ON intercoms_st_asgard_ch.intercom_uuid = intercoms_dir_asgard_ch.intercom_uuid
LEFT JOIN db1.intercoms_st_partner_ch AS intercoms_st_partner_ch 
	ON intercoms_st_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid
	AND intercoms_st_partner_ch.report_date = intercoms_st_asgard_ch.report_date
LEFT JOIN db1.intercoms_dir_partner_ch AS intercoms_dir_partner_ch ON intercoms_dir_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid
limit 10					
"""

ch.query_run(query_text)

,report_date,device_uuid,serial_number,type,asgard_partner_uuid,partner_uuid,installation_point_id,model_identifier
0,2023-07-14,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
1,2023-07-18,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
2,2023-07-15,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
3,2023-07-13,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
4,2023-07-16,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
5,2023-07-20,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
6,2023-07-21,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
7,2023-07-26,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
8,2023-07-24,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,
9,2023-07-19,0001e60b-206f-4047-a5c0-79b6f06f22b2,2209138569,intercom,,,0,


In [15]:
query_text = """--sql
SELECT
    --LAST_VALUE(cspc.installation_point_id) IGNORE NULLS OVER (PARTITION BY COALESCE(cspc.intercom_uuid,cspc.camera_uuid)  ORDER BY cspc.report_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS last_installation_point_id,
    --cspc.report_date as report_date,
    --COALESCE(cspc.intercom_uuid, cspc.camera_uuid) AS device_uuid,
    --COALESCE(motherboard_id, camera_serial) AS serial_number,
    IF(cspc.intercom_uuid = "","intercom","") AS device_type,
    cspc.partner_uuid as partner_uuid,
    cspc.installation_point_id as installation_point_id,
    ROW_NUMBER() OVER(PARTITION BY cspc.intercom_uuid, cspc.report_date ORDER BY cspc.installation_point_id DESC) AS bug_filter
FROM db1.cameras_st_partner_ch AS cspc
LEFT JOIN db1.installation_point_st_partner_ch  AS  ipcp
    ON cspc.installation_point_id = ipcp.installation_point_id
    AND cspc.report_date = ipcp.report_date
LEFT JOIN db1.cameras_dir_asgard_ch AS cdac
    ON cspc.camera_uuid = cdac.camera_uuid
LEFT JOIN db1.intercoms_dir_asgard_ch AS idac
    ON cspc.intercom_uuid = idac.intercom_uuid
WHERE cspc.`report_date`> DATE('2023-11-12')
LIMIT 10
"""

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 62
 Code: 62. DB::Exception: Syntax error: failed at position 445 ("") (line 7, col 29): "","intercom","") AS device_type,
    cspc.partner_uuid as partner_uuid,
    cspc.installation_point_id as installation_point_id,
    ROW_NUMBER() OVER(PARTITION B... Expected one of: token, Comma, ClosingRoundBracket, CAST operator, ANY, ALL, NOT, INTERVAL, CASE, DATE, TIMESTAMP, tuple, collection of literals, array, number, literal, NULL, NULL, Bool, TRUE, FALSE, string literal, asterisk, qualified asterisk, compound identifier, identifier, COLUMNS matcher, COLUMNS, qualified COLUMNS matcher, function name, substitution, MySQL-style global variable. (SYNTAX_ERROR) (version 25.3.3.42 (official build))


In [ ]:
query_text = """--sql
-- Соединяем добавляем данные из справочников
WITH intercoms_t AS (SELECT 
	intercoms_st_asgard_ch.report_date as report_date,
	intercoms_st_asgard_ch.intercom_uuid AS device_uuid,
	motherboard_id AS serial_number,
	'intercom' AS type,
	intercoms_st_asgard_ch.partner_uuid AS asgard_partner_uuid,
	intercoms_st_partner_ch.partner_uuid AS partner_uuid,
	intercoms_st_partner_ch.installation_point_id AS installation_point_id,
	intercoms_st_partner_ch.model_identifier as model_identifier
FROM db1.intercoms_st_asgard_ch AS intercoms_st_asgard_ch
LEFT JOIN db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard_ch ON intercoms_st_asgard_ch.intercom_uuid = intercoms_dir_asgard_ch.intercom_uuid
LEFT JOIN db1.intercoms_st_partner_ch AS intercoms_st_partner_ch 
	ON intercoms_st_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid
	AND intercoms_st_partner_ch.report_date = intercoms_st_asgard_ch.report_date
LEFT JOIN db1.intercoms_dir_partner_ch AS intercoms_dir_partner_ch ON intercoms_dir_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid							
),
--Добавляем данные из истории устройст. Удаляем ковычки, мешающие работе
device_history_clean AS(SELECT 
	date_of_changing, 
    number_of_changing, 
    replace(text_of_changing, '''', '') AS text_of_changing,
    replace(serial_number, '''', '') AS serial_number,
    replace(device_type, '''', '') AS device_type,
    replace(device_uuid, '''', '') AS device_uuid,
    replace(installation_point_id, '''', '') AS installation_point_id,
    replace(partner_uuid, '''', '') AS partner_uuid,
    replace(partner_uuid_lag, '''', '') AS partner_uuid_lag
FROM db1.device_history)
--
SELECT 
	report_date,
	intercoms_t.partner_uuid AS partner_uuid,
	type,
	number_of_changing,
	text_of_changing,
	installation_point_id,
	device_uuid,
	intercoms_t.serial_number as serial_number
FROM intercoms_t
LEFT JOIN device_history_clean ON device_history_clean.serial_number = intercoms_t.serial_number
							AND device_history_clean.date_of_changing = intercoms_t.report_date
limit 10
"""

ch.query_run(query_text)

In [ ]:
SELECT
	report_date,
	partner_uuid,
	installation_point_id,
	camera_uuid,
	intercom_uuid,
	IF(n_row = 1,1,0) AS first_appearence,
	IF(n_row != 1 AND lag_partner_uuid != partner_uuid,1,0) AS partner_uuid_change,
	IF(n_row != 1 AND lag_installation_point_id != 0 AND installation_point_id = 0,1,0) AS installation_point_removing,
	IF(n_row != 1 AND lag_installation_point_id = 0 AND installation_point_id != 0 AND installation_point_id != last_installation_point_id,1,0) AS installation_point_installation,
	IF(n_row != 1 AND last_installation_point_id = 0 AND installation_point_id != 0,1,0) AS installation_point_first_installing,
	IF(n_row != 1 AND installation_point_id !=0 AND lag_installation_point_id != installation_point_id,1,0) AS installation_pont_changing
FROM
	(SELECT
		report_date,
		partner_uuid,
		installation_point_id,
		intercom_uuid,
		camera_uuid,
		lagInFrame(installation_point_id,1,0) OVER (PARTITION BY camera_uuid ORDER BY report_date) AS lag_installation_point_id,
		lagInFrame(partner_uuid,1,'') OVER (PARTITION BY camera_uuid ORDER BY report_date) AS lag_partner_uuid,
		anyLast(installation_point_id) OVER (PARTITION BY camera_uuid ORDER BY report_date) AS last_installation_point_id,
		ROW_NUMBER() OVER (PARTITION by camera_uuid order by report_date) as n_row
	FROM db1.cameras_st_partner_ch)

In [7]:
query_text = """--sql
-- Соединяем добавляем данные из справочников
WITH intercoms_t AS (SELECT 
	intercoms_st_asgard_ch.report_date as report_date,
	intercoms_st_asgard_ch.intercom_uuid AS device_uuid,
	motherboard_id AS serial_number,
	'intercom' AS type,
	intercoms_st_asgard_ch.partner_uuid AS asgard_partner_uuid,
	intercoms_st_partner_ch.partner_uuid AS partner_uuid,
	intercoms_st_partner_ch.installation_point_id AS installation_point_id,
	intercoms_st_partner_ch.model_identifier as model_identifier
FROM db1.intercoms_st_asgard_ch AS intercoms_st_asgard_ch
LEFT JOIN db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard_ch ON intercoms_st_asgard_ch.intercom_uuid = intercoms_dir_asgard_ch.intercom_uuid
LEFT JOIN db1.intercoms_st_partner_ch AS intercoms_st_partner_ch 
	ON intercoms_st_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid
	AND intercoms_st_partner_ch.report_date = intercoms_st_asgard_ch.report_date
LEFT JOIN db1.intercoms_dir_partner_ch AS intercoms_dir_partner_ch ON intercoms_dir_partner_ch.intercom_uuid = intercoms_st_asgard_ch.intercom_uuid							
),
--Добавляем данные из истории устройст. Удаляем ковычки, мешающие работе
device_history_clean AS(SELECT 
	date_of_changing, 
    number_of_changing, 
    replace(text_of_changing, '''', '') AS text_of_changing,
    replace(serial_number, '''', '') AS serial_number,
    replace(device_type, '''', '') AS device_type,
    replace(device_uuid, '''', '') AS device_uuid,
    replace(installation_point_id, '''', '') AS installation_point_id,
    replace(partner_uuid, '''', '') AS partner_uuid,
    replace(partner_uuid_lag, '''', '') AS partner_uuid_lag
FROM db1.device_history),
--
intercoms_and_history AS(SELECT 
	report_date,
	intercoms_t.partner_uuid AS partner_uuid,
	type,
	number_of_changing,
	text_of_changing,
	installation_point_id,
	device_uuid ,
	intercoms_t.serial_number as serial_number
FROM intercoms_t
LEFT JOIN device_history_clean ON device_history_clean.serial_number = intercoms_t.serial_number
							AND device_history_clean.date_of_changing = intercoms_t.report_date
)
--
SELECT 
	report_date,
	partner_uuid,
	type,
	count(serial_number)
	--count(if(text_of_changing LIKE '%Первое упоминание%',serial_number,Null)) AS count_new_serial_number
	--count(if(text_of_changing LIKE '%Возвращение на точку установки%',serial_number,Null)) AS count_serial_number_returned,
	--count(if(text_of_changing LIKE '%Изменение партнера%',serial_number,Null)) AS count_serial_number_partner_change,
	--count(if(text_of_changing LIKE '%Cнятие с точки установки%',serial_number,Null)) AS count_serial_number_removal,
	--count(if(text_of_changing LIKE '%Изменение точки установки%',serial_number,Null)) AS count_serial_number_installation_point_change,
	--count(if((text_of_changing LIKE '%Первая установка%') 
	--		OR (text_of_changing LIKE '%Первое упоминание%' AND installation_point_id != 0),serial_number,Null)) AS count_serial_number_first_installation,
	--count(if(partner_uuid != '',serial_number,Null)) AS count_serial_number,
    --count(if( installation_point_id != 0,serial_number,Null)) AS count_serial_number_on_installation_point
FROM
	intercoms_and_history
GROUP BY
	report_date,
	partner_uuid,
	type
limit 100
"""
ch.query_run(query_text)

Unexpected Http Driver Exception


OperationalError: Error HTTPSConnectionPool(host='rc1a-eflrt968scbm232q.mdb.yandexcloud.net', port=8443): Read timed out. (read timeout=10) executing HTTP request attempt 1 (https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443)

In [2]:

query_text = """--sql
    SELECT
        order_date,
        partner_lk,
        type_simple,
        sum(count_in_order) count_in_order,
        sum(sale_sum) AS sale_sum
    FROM 
        (SELECT 
            order_date,
            partner_lk,
            count_in_order,
            sale_sum,
            CASE
                WHEN model = 'ГОС' OR model = 'ГОС 22' OR model = 'ГОС БР' THEN 'ГОС'
                WHEN model = 'ИО' OR model = 'ИО 22' OR model = 'ИО АПИ' OR model = 'ИО ПРО' THEN 'ИО'
                ElSE 'Прочие модели'
            END AS type_simple
        FROM db1.all_order_google_sheets
        WHERE type = 'Домофоны'
        )
    GROUP BY partner_lk,
            order_date,
            type_simple
    """
ch.query_run(query_text)

,order_date,partner_lk,type_simple,count_in_order,sale_sum
0,2024-07-22,132066,ИО,3,44400
1,2024-01-03,123945,ИО,1,14800
2,2024-10-31,121988,Прочие модели,4,59200
3,2024-01-23,123231,ИО,4,50000
4,2025-01-17,124731,ГОС,7,99600
...,...,...,...,...,...
320,2024-01-18,120506,ИО,6,88800
321,2024-09-27,123905,Прочие модели,1,14800
322,2024-01-24,144666,ИО,5,66000
323,2024-12-02,120567,ИО,12,177600


In [ ]:
query_text = """--sql
    WITH all_order_intercomes AS(
        SELECT
            order_date,
            partner_lk,
            type_simple,
            sum(count_in_order) count_in_order,
            sum(sale_sum) AS sale_sum
        FROM 
            (SELECT 
                order_date,
                partner_lk,
                count_in_order,
                sale_sum,
                CASE
                    WHEN model = 'ГОС' OR model = 'ГОС 22' OR model = 'ГОС БР' THEN 'ГОС'
                    WHEN model = 'ИО' OR model = 'ИО 22' OR model = 'ИО АПИ' OR model = 'ИО ПРО' THEN 'ИО'
                    ElSE 'Прочие модели'
                END AS type_simple
            FROM db1.all_order_google_sheets
            WHERE type = 'Домофоны'
            )
        GROUP BY partner_lk,
                order_date,
                type_simple),
    --
        all_order_with_accumulated_sum AS(
        SELECT
            order_date,
            if(partner_lk = 0,Null,partner_lk) as partner_lk,
            type_simple,
            count_in_order,
            sale_sum,
            sum(count_in_order) OVER (PARTITION BY partner_lk,type_simple ORDER BY order_date) AS accumulated_count,
            sum(sale_sum) OVER (PARTITION BY partner_lk,type_simple ORDER BY order_date) AS accumulated_sum
        FROM all_order_intercomes
        ),
    --			
        date_range_cross_join AS (
        SELECT 
            date_range,	
            partner_lk,
            type_simple
        FROM db1.date_range_table
        CROSS JOIN 
            (SELECT
                DISTINCT
                partner_lk,
                type_simple
            FROM all_order_with_accumulated_sum) as all_order_with_accumulated_sum
        ),
    --
        date_range_with_orders AS (SELECT
            date_range,
            if(partner_lk = 0,Null,partner_lk) as partner_lk,
            if(type_simple = '',Null, type_simple) as type_simple,
            if(count_in_order = 0,Null,count_in_order) as count_in_order,
            if(accumulated_count = 0,Null,accumulated_count) as accumulated_count,
            if(sale_sum = 0, Null,sale_sum) as sale_sum,
            if(accumulated_sum = 0,Null,accumulated_sum) AS accumulated_sum
        FROM date_range_cross_join
        LEFT JOIN all_order_with_accumulated_sum ON all_order_with_accumulated_sum.order_date = date_range_cross_join.date_range
                                                AND all_order_with_accumulated_sum.partner_lk = date_range_cross_join.partner_lk
                                                AND all_order_with_accumulated_sum.type_simple = date_range_cross_join.type_simple
        ORDER BY  type_simple,partner_lk, date_range)
    --
    SELECT
        date_range,
        partner_lk,
        type_simple,
    --	last_value(count_in_order) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as count_in_order,
        count_in_order,
        last_value(accumulated_count) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as accumulated_count,
        sale_sum,
    --	last_value(sale_sum) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as sale_sum,
        last_value(accumulated_sum) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as accumulated_sum
    FROM date_range_with_orders
    ORDER BY  type_simple,partner_lk, date_range
    """

ch.query_run(query_text)

In [ ]:

INSERT INTO FUNCTION s3(
    'https://storage.yandexcloud.net/aggregated-data/all_order_accumulated_data/all_order_accumulated_data.parquet',
    'parquet'
)
SETTINGS s3_truncate_on_insert = 1
WITH all_order_intercomes AS(
	SELECT
		order_date,
		partner_lk,
		type_simple,
		sum(count_in_order) count_in_order,
		sum(sale_sum) AS sale_sum
	FROM 
		(SELECT 
			order_date,
			partner_lk,
			count_in_order,
			sale_sum,
			CASE
				WHEN model = 'ГОС' OR model = 'ГОС 22' OR model = 'ГОС БР' THEN 'ГОС'
				WHEN model = 'ИО' OR model = 'ИО 22' OR model = 'ИО АПИ' OR model = 'ИО ПРО' THEN 'ИО'
				ElSE 'Прочие модели'
			END AS type_simple
		FROM db1.all_order_google_sheets
		WHERE type = 'Домофоны'
		)
	GROUP BY partner_lk,
			order_date,
			type_simple),
--
	all_order_with_accumulated_sum AS(
	SELECT
		order_date,
		if(partner_lk = 0,Null,partner_lk) as partner_lk,
		type_simple,
		count_in_order,
		sale_sum,
		sum(count_in_order) OVER (PARTITION BY partner_lk,type_simple ORDER BY order_date) AS accumulated_count,
		sum(sale_sum) OVER (PARTITION BY partner_lk,type_simple ORDER BY order_date) AS accumulated_sum
	FROM all_order_intercomes
	),
--			
	date_range_cross_join AS (
	SELECT 
		date_range,	
		partner_lk,
		type_simple
	FROM db1.date_range_table
	CROSS JOIN 
		(SELECT
			DISTINCT
			partner_lk,
			type_simple
		FROM all_order_with_accumulated_sum) as all_order_with_accumulated_sum
	),
--
	date_range_with_orders AS (SELECT
		date_range,
		if(partner_lk = 0,Null,partner_lk) as partner_lk,
		if(type_simple = '',Null, type_simple) as type_simple,
		if(count_in_order = 0,Null,count_in_order) as count_in_order,
		if(accumulated_count = 0,Null,accumulated_count) as accumulated_count,
		if(sale_sum = 0, Null,sale_sum) as sale_sum,
		if(accumulated_sum = 0,Null,accumulated_sum) AS accumulated_sum
	FROM date_range_cross_join
	LEFT JOIN all_order_with_accumulated_sum ON all_order_with_accumulated_sum.order_date = date_range_cross_join.date_range
											AND all_order_with_accumulated_sum.partner_lk = date_range_cross_join.partner_lk
											AND all_order_with_accumulated_sum.type_simple = date_range_cross_join.type_simple
	ORDER BY  type_simple,partner_lk, date_range)
--
SELECT
	date_range,
	partner_lk,
	type_simple,
--	last_value(count_in_order) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as count_in_order,
	count_in_order,
	last_value(accumulated_count) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as accumulated_count,
	sale_sum,
--	last_value(sale_sum) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as sale_sum,
	last_value(accumulated_sum) IGNORE NULLS OVER (partition by partner_lk,type_simple ORDER BY date_range) as accumulated_sum
FROM date_range_with_orders
ORDER BY  type_simple,partner_lk, date_range

